In [1]:
#load libraries
#imports
#imports 
import os
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
import requests
from bs4 import BeautifulSoup
import lxml
import html5lib
import urllib
import string
import re
import math
import time
import tqdm
import datetime

In [2]:
#directory structure for inputs/outputs
class DirStructure(object):
    """
    Create directory and file structure.
    """
    def __init__(self, code_directory,results_folder,results_case):
        self.DIRECTORY = code_directory
        self.FORMATTED_INPUTS_DIRECTORY = os.path.join(self.DIRECTORY, "FormattedData")
        self.SOURCE_INPUTS_DIRECTORY = os.path.join(self.DIRECTORY, "SourceData")
        self.TIMESERIES_INPUTS_DIRECTORY = os.path.join(self.DIRECTORY, "timeseries_data_files")
        self.HEATRATE_INPUTS_DIRECTORY = os.path.join(self.SOURCE_INPUTS_DIRECTORY,"HR_data")
        
        self.CASE_DIRECTORY = os.path.join(os.getcwd(),results_folder)
        self.RESULTS_DIRECTORY = os.path.join(os.getcwd(), results_folder+'\\'+results_case)
        self.RESULTS_INPUTS_DIRECTORY = os.path.join(self.RESULTS_DIRECTORY,'inputs')

    def make_directories(self):
        if not os.path.exists(self.CASE_DIRECTORY):
            os.mkdir(self.CASE_DIRECTORY)
        if not os.path.exists(self.RESULTS_DIRECTORY):
            os.mkdir(self.RESULTS_DIRECTORY)
        if not os.path.exists(self.RESULTS_INPUTS_DIRECTORY):
            os.mkdir(self.RESULTS_INPUTS_DIRECTORY)
            
f = DirStructure('C:\\Users\\llavin\\Desktop\\luke_mpec\\RTS-GMLC-master\\RTS_Data','base','hi')
f.make_directories()

In [3]:
#nrel rts file structure and data loads
#folders.TIMESERIES_INPUTS_DIRECTORY

#SourceData folder
branch_data = pd.read_csv(os.path.join(f.SOURCE_INPUTS_DIRECTORY,'branch.csv'))
bus_data = pd.read_csv(os.path.join(f.SOURCE_INPUTS_DIRECTORY,'bus.csv'))
dcbranch_data = pd.read_csv(os.path.join(f.SOURCE_INPUTS_DIRECTORY,'dc_branch.csv'))
gen_data = pd.read_csv(os.path.join(f.SOURCE_INPUTS_DIRECTORY,'gen.csv'))
reserves_data = pd.read_csv(os.path.join(f.SOURCE_INPUTS_DIRECTORY,'reserves.csv'))
simulationobjects_data =pd.read_csv(os.path.join(f.SOURCE_INPUTS_DIRECTORY,'simulation_objects.csv'))
storage_data = pd.read_csv(os.path.join(f.SOURCE_INPUTS_DIRECTORY,'storage.csv'))
timeseriespointers_data = pd.read_csv(os.path.join(f.SOURCE_INPUTS_DIRECTORY,'timeseries_pointers.csv'))

## Timeseries folder ##

#generation
hydro_data = pd.read_csv(os.path.join(f.TIMESERIES_INPUTS_DIRECTORY,'Hydro\\DAY_AHEAD_hydro.csv'))
load_data = pd.read_csv(os.path.join(f.TIMESERIES_INPUTS_DIRECTORY,'Load\\DAY_AHEAD_regional_Load.csv'))
pv_data = pd.read_csv(os.path.join(f.TIMESERIES_INPUTS_DIRECTORY,'PV\\DAY_AHEAD_pv.csv'))
rtpv_data = pd.read_csv(os.path.join(f.TIMESERIES_INPUTS_DIRECTORY,'RTPV\\DAY_AHEAD_rtpv.csv'))
csp_data = pd.read_csv(os.path.join(f.TIMESERIES_INPUTS_DIRECTORY,'CSP\\DAY_AHEAD_Natural_Inflow.csv'))
wind_data = pd.read_csv(os.path.join(f.TIMESERIES_INPUTS_DIRECTORY,'Wind\\DAY_AHEAD_Wind.csv'))

#reserves
reg_up_data = pd.read_csv(os.path.join(f.TIMESERIES_INPUTS_DIRECTORY,'Reserves\\DAY_AHEAD_regional_Reg_Up.csv'))
reg_down_data = pd.read_csv(os.path.join(f.TIMESERIES_INPUTS_DIRECTORY,'Reserves\\DAY_AHEAD_regional_Reg_Down.csv'))
flex_up_data = pd.read_csv(os.path.join(f.TIMESERIES_INPUTS_DIRECTORY,'Reserves\\DAY_AHEAD_regional_Flex_Up.csv'))
flex_down_data = pd.read_csv(os.path.join(f.TIMESERIES_INPUTS_DIRECTORY,'Reserves\\DAY_AHEAD_regional_Flex_Down.csv'))

## redo of create files as oop with inputs



In [21]:
class CreateRTSCase(object):
    def __init__(self,gentypes,gen_data,storage_data,hydro_data,csp_data,rtpv_data,pv_data,wind_data,
                 bus_data,branch_data,reg_up_data,reg_down_data,flex_up_data,flex_down_data,
                 directory,hour_begin,hours=24,lb_to_tonne=0.000453592,baseMVA = 100,km_per_mile = 1.60934):
        self.gentypes = gentypes
        self.gen_data = gen_data
        self.storage_data = storage_data
        self.hydro_data = hydro_data
        self.csp_data = csp_data
        self.rtpv_data = rtpv_data
        self.pv_data = pv_data
        self.wind_data = wind_data
        self.bus_data = bus_data
        self.branch_data = branch_data
        self.reg_up_data = reg_up_data
        self.reg_down_data = reg_down_data
        self.flex_up_data = flex_up_data
        self.flex_down_data = flex_down_data
        
        self.directory = directory
        #self.directory.RESULTS_DIRECTORY = os.path.join(os.getcwd(), "test\\"+case_folder)
        #self.RESULTS_DIRECTORY = os.path.join(self.directory.CASE_DIRECTORY, case_folder)
        #self.directory.RESULTS_INPUTS_DIRECTORY = os.path.join(self.directory.RESULTS_DIRECTORY,'inputs')
        #self.directory.make_directories()
        
        self.hour_begin = hour_begin
        self.hours = hours
        self.hour_end = hour_begin+hours
        self.lb_to_tonne = lb_to_tonne
        self.baseMVA = baseMVA
        self.km_per_mile = km_per_mile
    
    def dict_to_csv(self,filename,mydict,index=['Gen_Index'],owned_gens=False):
        df = pd.DataFrame.from_dict(mydict)
        df.set_index(index,inplace=True)
        if owned_gens:
            for gen in self.owned_gen_list:
                df.at[gen,'GencoIndex'] = 1 #overwrite to make competitive
        df.to_csv(os.path.join(self.directory.RESULTS_INPUTS_DIRECTORY,filename+'.csv'))
    
    def df_to_csv(self,filename,mydict):
        df = pd.DataFrame(mydict,index=[0])
        df.set_index(['Storage_Index'],inplace=True)
        df.to_csv(os.path.join(self.directory.RESULTS_INPUTS_DIRECTORY,filename+'.csv'))
        
    def create_index(self):
        case_index_df = pd.DataFrame({'genco':[1],
                            '':['index']})
        case_index_df.set_index([''], inplace=True)
        case_index_df.to_csv(os.path.join(self.directory.RESULTS_INPUTS_DIRECTORY,'case_index.csv'))
        
    def pjm_generators(self,filename,owned_gen_list=[]):
        
        self.pjm_generators_dict = {}
        self.owned_gen_list = owned_gen_list
        index_list = ['Gen_Index','Capacity',"Fuel_Cost",'Pmin','start_cost','Can_Spin','Can_NonSpin',
                              'Min_Up','Min_Down','No_Load_Cost','Ramp_Rate',
              'tonneCO2perMWh','CO2price','CO2dollarsperMWh','ZoneLabel','GencoIndex']
        
        self.pjm_generators_dict[index_list[0]] = self.gen_data[self.gen_data['Unit Type'].isin(self.gentypes)]['GEN UID'].values
        self.pjm_generators_dict[index_list[1]] = self.gen_data[self.gen_data['Unit Type'].isin(self.gentypes)]['PMax MW'].values
        gen_data['$/MWH'] = gen_data['Fuel Price $/MMBTU']*self.gen_data['HR_incr_3']/1000
        self.pjm_generators_dict[index_list[2]] = self.gen_data[self.gen_data['Unit Type'].isin(self.gentypes)]['$/MWH'].values
        #defaults for now
        self.pjm_generators_dict[index_list[3]] = [0]*len(self.pjm_generators_dict[index_list[0]])
        self.pjm_generators_dict[index_list[4]] = [0]*len(self.pjm_generators_dict[index_list[0]])
        self.pjm_generators_dict[index_list[5]] = [1]*len(self.pjm_generators_dict[index_list[0]])
        self.pjm_generators_dict[index_list[6]] = [1]*len(self.pjm_generators_dict[index_list[0]])
        self.pjm_generators_dict[index_list[7]] = [1]*len(self.pjm_generators_dict[index_list[0]])
        self.pjm_generators_dict[index_list[8]] = [1]*len(self.pjm_generators_dict[index_list[0]])
        self.pjm_generators_dict[index_list[9]] = [0]*len(self.pjm_generators_dict[index_list[0]])
        #ramp rates
        self.pjm_generators_dict[index_list[10]] = 60*self.gen_data[self.gen_data['Unit Type'].isin(self.gentypes)]['Ramp Rate MW/Min'].values

        self.gen_data['CO2/MWH'] = self.gen_data['Emissions CO2 Lbs/MMBTU']*self.lb_to_tonne*self.gen_data['HR_incr_3']/1000
        self.pjm_generators_dict[index_list[11]] = self.gen_data[self.gen_data['Unit Type'].isin(self.gentypes)]['CO2/MWH'].values
        self.pjm_generators_dict[index_list[12]] = [0]*len(self.pjm_generators_dict[index_list[0]])
        self.pjm_generators_dict[index_list[13]] = [a*b for a,b in zip(self.pjm_generators_dict[index_list[11]],self.pjm_generators_dict[index_list[12]])]
        self.pjm_generators_dict[index_list[14]] = self.gen_data[self.gen_data['Unit Type'].isin(self.gentypes)]['Bus ID'].values
        self.pjm_generators_dict[index_list[15]] = [2]*len(self.pjm_generators_dict[index_list[0]])
        #for gen in owned_gen_list:  
        
        self.dict_to_csv(filename,self.pjm_generators_dict,owned_gens=True)
    
    def pjm_generators_full(self,filename):
        d = {}
        index_list = ['Gen_Index','Name','Zone','Category','Capacity',
              'Fuel_Cost','Can_Spin','UTILUNIT']
        d[index_list[0]] = self.pjm_generators_dict['Gen_Index']
        d[index_list[1]] = self.pjm_generators_dict['Gen_Index']
        d[index_list[2]] = self.pjm_generators_dict['ZoneLabel']
        d[index_list[3]] = self.gen_data[self.gen_data['Unit Type'].isin(self.gentypes)]['Category'].values
        d[index_list[4]] = self.pjm_generators_dict['Capacity']
        d[index_list[5]] = self.pjm_generators_dict['Fuel_Cost']
        d[index_list[6]] = self.pjm_generators_dict['Can_Spin']
        d[index_list[7]] = ['NA']*len(self.pjm_generators_dict[index_list[0]])
        
        self.dict_to_csv(filename,d)
        
    def storage(self,filename,size_scalar=1,busID=0):
        storage_dict = {}
        index_list = ['Storage_Index','Discharge','Charge','SOCMax','DischargeEff','ChargeEff',
                      'StorageZoneLabel','StorageIndex']

        storage_dict[index_list[0]] = self.gen_data[self.gen_data['Unit Type']=='STORAGE'].reset_index().at[0,'GEN UID']
        storage_dict[index_list[1]] = size_scalar*self.gen_data[self.gen_data['Unit Type']=='STORAGE'].reset_index().at[0,'PMax MW']
        storage_dict[index_list[2]] = size_scalar*self.gen_data[self.gen_data['Unit Type']=='STORAGE'].reset_index().at[0,'PMax MW']
        storage_dict[index_list[3]] = size_scalar*self.storage_data.at[1,'Max Volume GWh']*1000
        storage_dict[index_list[4]] = 1
        storage_dict[index_list[5]] = 1
        if busID==0:
            storage_dict[index_list[6]] = self.gen_data[self.gen_data['Unit Type']=='STORAGE'].reset_index().at[0,'Bus ID']
        else:
            storage_dict[index_list[6]] = int(busID)
        storage_dict[index_list[7]] = 2
        
        self.df_to_csv(filename,storage_dict)
    def init_gens(self,filename):
        d = {}
        index_list = ['Gen_Index','commit_init','time_up_init','time_down_init']

        d[index_list[0]] = self.pjm_generators_dict['Gen_Index']
        d[index_list[1]] = [1]*len(d[index_list[0]])
        d[index_list[2]] = [200]*len(d[index_list[0]])
        d[index_list[3]] = [0]*len(d[index_list[0]])
        self.dict_to_csv(filename,d)
        
    def scheduled_gens(self,filename):
        scheduled_dict = {}
        index_list = ['timepoint','Gen_Index','available','Capacity',"Fuel_Cost"]

        scheduled_dict[index_list[0]] = [e for e in list(range(1,self.hours+1)) for i in range(len(self.pjm_generators_dict['Gen_Index']))] 
        scheduled_dict[index_list[1]] = list(self.pjm_generators_dict['Gen_Index'])*self.hours
        scheduled_dict[index_list[2]] = [1]*len(scheduled_dict[index_list[1]])

        gen_cap_list = []
        for h in range(self.hour_begin,self.hour_end):
            for gen,capacity in zip(self.pjm_generators_dict['Gen_Index'],self.pjm_generators_dict['Capacity']):
                if gen in self.hydro_data.columns:
                    gen_cap_list.append(self.hydro_data.at[h,gen])
                elif gen in self.pv_data.columns:
                    gen_cap_list.append(self.pv_data.at[h,gen])
                elif gen in self.rtpv_data.columns:
                    gen_cap_list.append(self.rtpv_data.at[h,gen])
                elif gen in self.csp_data.columns:
                    gen_cap_list.append(self.csp_data.at[h,gen])
                elif gen in self.wind_data.columns:
                    gen_cap_list.append(self.wind_data.at[h,gen])
                else:
                    gen_cap_list.append(capacity)
        scheduled_dict[index_list[3]] = gen_cap_list
        scheduled_dict[index_list[4]] = list(self.pjm_generators_dict['Fuel_Cost'])*self.hours
        self.dict_to_csv(filename,scheduled_dict,index='timepoint')
        
    def timepoints(self,filename):
        self.timepoint_dict = {}
        index_list = ['timepoint','reference_bus','reg_up_mw','reg_down_mw',
             'flex_up_mw','flex_down_mw']

        self.timepoint_dict[index_list[0]] = list(range(1,self.hours+1))
        self.timepoint_dict[index_list[1]] = [self.bus_data.at[12,'Bus ID']]*len(self.timepoint_dict[index_list[0]])

        #reformatting for timepoints
        period_list = []
        for h in range(self.hour_begin,self.hour_end):
            if h==self.hour_begin:
                month = self.hydro_data.at[h,'Month']
                day = self.hydro_data.at[h,'Day']
            period_list.append(str(self.hydro_data.at[h,'Period']))
        self.timepoint_dict[index_list[2]] = list(self.reg_up_data[(self.reg_up_data['Month']==month) & (self.reg_up_data['Day']==day)][period_list].values.ravel())
        self.timepoint_dict[index_list[3]] = list(self.reg_down_data[(self.reg_down_data['Month']==month) & (self.reg_down_data['Day']==day)][period_list].values.ravel())
        self.timepoint_dict[index_list[4]] = list(self.flex_up_data[(self.flex_up_data['Month']==month) & (self.flex_up_data['Day']==day)][period_list].values.ravel())
        self.timepoint_dict[index_list[5]] = list(self.flex_down_data[(self.flex_down_data['Month']==month) & (self.flex_down_data['Day']==day)][period_list].values.ravel())
        self.dict_to_csv(filename,self.timepoint_dict,index='timepoint')
        
    def zones(self,filename):
        self.zone_dict = {}
        index_list = ['zone','wind_cap','solar_cap','voltage_angle_max','voltage_angle_min']

        self.zone_dict[index_list[0]] = self.bus_data.loc[:,'Bus ID'].values
        self.zone_dict[index_list[1]] = [0]*len(self.zone_dict[index_list[0]]) #eventually replace this
        self.zone_dict[index_list[2]] = [0]*len(self.zone_dict[index_list[0]]) #eventually replace this
        self.zone_dict[index_list[3]] = [180]*len(self.zone_dict[index_list[0]]) #math.pi/3
        self.zone_dict[index_list[4]] = [-180]*len(self.zone_dict[index_list[0]]) #math.pi/3
        self.dict_to_csv(filename,self.zone_dict,index='zone')
        
    def zonal_loads(self,filename):
        bus_df = self.bus_data[['Bus ID','MW Load','Area']]
        bus_df_load = pd.merge(bus_df,bus_df.groupby('Area').sum()[['MW Load']].reset_index(),how='left',left_on='Area',right_on='Area')
        bus_df_load['Frac Load'] = bus_df_load['MW Load_x']/bus_df_load['MW Load_y']
        hourly_df = pd.concat([bus_df_load]*self.hours, ignore_index=True)
        hourly_df['timepoint'] = [e for e in self.timepoint_dict['timepoint'] for i in self.zone_dict['zone']]
        #print(load_data)
        load_short = load_data.loc[self.hour_begin:self.hour_end-1].set_index(['Month','Day','Period']).reset_index() #sets unique index
        hourly_df['zonal_load'] = [load_short.at[t-1,str(a)] for t,a in zip(hourly_df['timepoint'].values,hourly_df['Area'].values)]
        hourly_df['bus load'] = hourly_df['zonal_load']*hourly_df['Frac Load']
        
        time_zone_dict = {}
        index_list = ['timepoint','zone','gross_load','wind_cf','solar_cf']

        time_zone_dict[index_list[0]] = [e for e in self.timepoint_dict['timepoint'] for i in self.zone_dict['zone']] 
        time_zone_dict[index_list[1]] = [i for e in self.timepoint_dict['timepoint'] for i in self.zone_dict['zone']]
        time_zone_dict[index_list[2]] = hourly_df['bus load'].values#hourly_df['MW Load_x'].values
        time_zone_dict[index_list[3]] = [0]*len(time_zone_dict[index_list[0]])
        time_zone_dict[index_list[4]] = [0]*len(time_zone_dict[index_list[0]])

        self.dict_to_csv(filename,time_zone_dict,index='timepoint')
        
    def tx_lines(self,filename):

        branch_data_update = pd.merge(self.branch_data,self.bus_data[['Bus ID','BaseKV']],how='left',left_on='From Bus',right_on='Bus ID')
        branch_data_update['puconversion'] = branch_data_update.BaseKV**2/self.baseMVA #V^2/P=R, kv^2 and MV will cancel units
        branch_data_update['Lengthkm'] =branch_data_update['Length']*self.km_per_mile 
        for i in branch_data_update.index:
            if branch_data_update.at[i,'B']==0:
                branch_data_update.at[i,'puconversion'] = branch_data_update.at[i,'Tr Ratio']
                branch_data_update.at[i,'Lengthkm'] = 1
            else:
                branch_data_update.at[i,'puconversion'] = 1
        branch_data_update['x_final']=branch_data_update.X*branch_data_update.puconversion#*branch_data_update.Lengthkm
        branch_data_update['r_final']=branch_data_update.R*branch_data_update.puconversion#*branch_data_update.Lengthkm
        self.branch_data_final = branch_data_update#[branch_data_update['Lengthkm']!=0].copy()
        
        self.branch_data_final['Susceptance'] = [1/x*100*math.pi/180 for x in self.branch_data_final['x_final'].values]
        self.tx_dict = {}
        index_list = ['transmission_line','susceptance']

        self.tx_dict[index_list[0]] = self.branch_data_final['UID'].values
        self.tx_dict[index_list[1]] = self.branch_data_final['Susceptance'].values 
        self.dict_to_csv(filename,self.tx_dict,index='transmission_line')
    
    def tx_lines_hourly(self,filename,flow_multiplier=1):
        tx_hourly_dict = {}
        index_list = ['timepoint','transmission_line','transmission_from','transmission_to',
                      'min_flow','max_flow','hurdle_rate']

        tx_hourly_dict[index_list[0]] = [t for t in self.timepoint_dict['timepoint'] for line in self.tx_dict['transmission_line']]
        tx_hourly_dict[index_list[1]] = [line for t in self.timepoint_dict['timepoint'] for line in self.tx_dict['transmission_line']]
        tx_hourly_dict[index_list[2]] = [t_from for t in self.timepoint_dict['timepoint'] for t_from in self.branch_data_final['From Bus'].values]
        tx_hourly_dict[index_list[3]] = [t_to for t in self.timepoint_dict['timepoint'] for t_to in self.branch_data_final['To Bus'].values]
        tx_hourly_dict[index_list[4]] = [-flow_multiplier*flow for t in self.timepoint_dict['timepoint'] for flow in self.branch_data_final['Cont Rating'].values]
        tx_hourly_dict[index_list[5]] = [flow_multiplier*flow for t in self.timepoint_dict['timepoint'] for flow in self.branch_data_final['Cont Rating'].values]
        tx_hourly_dict[index_list[6]] = [0]*len(tx_hourly_dict[index_list[0]])
        self.dict_to_csv(filename,tx_hourly_dict,index=['timepoint','transmission_line'])
    
    def gs(self):
        self.gs_list = [0,1,2,3]
        gs_df = pd.DataFrame({'generator_segment':self.gs_list,
                            'length':[.4,.2,.2,.2]})
        gs_df.set_index(['generator_segment'], inplace=True)
        gs_df.to_csv(os.path.join(self.directory.RESULTS_INPUTS_DIRECTORY,'generator_segments.csv'))
        
    def gs_seg(self,CO2price=0):
        gs_seg_dict = {}
        index_list = ['time','Gen_Index','generator_segment','segment_length',
                      'marginal_cost','prev_offer','marginal_CO2','CO2damage']

        gs_seg_dict[index_list[0]] = [t for t in self.timepoint_dict['timepoint'] for gen in self.pjm_generators_dict['Gen_Index'] for gs in self.gs_list]
        gs_seg_dict[index_list[1]] = [gen for t in self.timepoint_dict['timepoint'] for gen in self.pjm_generators_dict['Gen_Index'] for gs in self.gs_list]
        gs_seg_dict[index_list[2]] = [gs for t in self.timepoint_dict['timepoint'] for gen in self.pjm_generators_dict['Gen_Index'] for gs in self.gs_list]
        
        mcos_list = []
        emissions_list = []
        seg_length_list = []
        for gen,gs in zip(gs_seg_dict[index_list[1]],gs_seg_dict[index_list[2]]):
            EmissionsRate = self.gen_data.set_index('GEN UID').at[gen,'Emissions CO2 Lbs/MMBTU']*self.lb_to_tonne
            if gs==0:
                seg_length_list.append(self.gen_data.set_index('GEN UID').at[gen,'Output_pct_'+str(gs)])
                if self.gen_data.set_index('GEN UID').at[gen,'HR_incr_'+str(1)]/1000!=0:
                    HeatRate = self.gen_data.set_index('GEN UID').at[gen,'HR_incr_'+str(1)]/1000 #convexify offer curve
                else:
                    HeatRate = self.gen_data.set_index('GEN UID').at[gen,'HR_avg_'+str(gs)]/1000
                mcos_list.append(HeatRate*self.gen_data.set_index('GEN UID').at[gen,'Fuel Price $/MMBTU'])
                emissions_list.append(HeatRate*EmissionsRate)
            else:
                seg_length_list.append(max(0,self.gen_data.set_index('GEN UID').at[gen,'Output_pct_'+str(gs)]-gen_data.set_index('GEN UID').at[gen,'Output_pct_'+str(gs-1)]))
                if self.gen_data.set_index('GEN UID').at[gen,'HR_incr_'+str(gs)]/1000==0 and self.gen_data.set_index('GEN UID').at[gen,'HR_avg_'+str(0)]/1000 !=0:
                    HeatRate = self.gen_data.set_index('GEN UID').at[gen,'HR_avg_'+str(0)]/1000
                else:
                    HeatRate = self.gen_data.set_index('GEN UID').at[gen,'HR_incr_'+str(gs)]/1000
                mcos_list.append(HeatRate*self.gen_data.set_index('GEN UID').at[gen,'Fuel Price $/MMBTU'])
                emissions_list.append(HeatRate*EmissionsRate)

        gs_seg_dict[index_list[3]] = seg_length_list
        gs_seg_dict[index_list[4]] = mcos_list
        gs_seg_dict[index_list[5]] = gs_seg_dict[index_list[4]]
        gs_seg_dict[index_list[6]] = emissions_list
        gs_seg_dict[index_list[7]] = [CO2price]*len(emissions_list)
        
        for i in range(len(gs_seg_dict[index_list[4]])):
            gs_seg_dict[index_list[4]][i] += gs_seg_dict[index_list[6]][i]*gs_seg_dict[index_list[7]][i]
        
        gs_seg_df = pd.DataFrame.from_dict(gs_seg_dict)
        gs_seg_df.set_index(['time','Gen_Index'],inplace=True)
        #overwrite 0's on segment 1 for renewable generators
        gs_seg_df.loc[(gs_seg_df.generator_segment == 0) & (gs_seg_df.segment_length == 0), 'segment_length'] = 1
        gs_seg_df.to_csv(os.path.join(self.directory.RESULTS_INPUTS_DIRECTORY,'generator_segment_marginalcost.csv'))
        #print(gs_seg_df)
        return gs_seg_df

In [25]:

#create folder structure


#run
start = datetime.datetime.strptime("01-01-2019", "%m-%d-%Y")
end = datetime.datetime.strptime("01-01-2020", "%m-%d-%Y")

date_generated = [start + datetime.timedelta(days=x) for x in range(0, (end-start).days)]
date_folders = [date.strftime("%m.%d.%Y") for date in date_generated]
#date_folders = ['1.1.2020']

for case_name, begin_hour in zip(date_folders, [i for i in range(0,len(date_folders)*24,24)]):
    f = DirStructure('C:\\Users\\Luke\\Documents\\GitHub\\RTS-GMLC-master\\RTS_Data',
                     'CompetitiveStorageBus309andWind303',case_name)
    f.make_directories()
    print(case_name)
    gt = ['CT','STEAM','CC','NUCLEAR','HYDRO','SYNC_COND','RTPV','WIND','PV','CSP']
    case = CreateRTSCase(gt,gen_data,storage_data,hydro_data,csp_data,rtpv_data,pv_data,wind_data,
                         bus_data,branch_data,reg_up_data,reg_down_data,flex_up_data,flex_down_data,
                         f,begin_hour)
    case.create_index()
    owned_gens = ['303_WIND_1']
    case.pjm_generators('PJM_generators',owned_gen_list = owned_gens)
    case.pjm_generators_full('PJM_generators_full')
    case.storage('storage_resources',busID=309)#size_scalar=0
    case.init_gens('initialize_generators')
    case.scheduled_gens('PJM_generators_scheduled_outage')
    case.timepoints('timepoints_index')
    case.zones('zones')
    case.zonal_loads('timepoints_zonal')
    case.tx_lines('transmission_lines')
    case.tx_lines_hourly('transmission_lines_hourly',flow_multiplier=1)#.75
    case.gs()
    #case.gs_seg() 
    try:
        p.to_csv(os.path.join(case.directory.RESULTS_INPUTS_DIRECTORY,'generator_segment_marginalcost.csv'))
    except NameError:
        print('hi')
        p = case.gs_seg(CO2price=0)

01.01.2019
hi
01.02.2019
01.03.2019
01.04.2019
01.05.2019
01.06.2019
01.07.2019
01.08.2019
01.09.2019
01.10.2019
01.11.2019
01.12.2019
01.13.2019
01.14.2019
01.15.2019
01.16.2019
01.17.2019
01.18.2019
01.19.2019
01.20.2019
01.21.2019
01.22.2019
01.23.2019
01.24.2019
01.25.2019
01.26.2019
01.27.2019
01.28.2019
01.29.2019
01.30.2019
01.31.2019
02.01.2019
02.02.2019
02.03.2019
02.04.2019
02.05.2019
02.06.2019
02.07.2019
02.08.2019
02.09.2019
02.10.2019
02.11.2019
02.12.2019
02.13.2019
02.14.2019
02.15.2019
02.16.2019
02.17.2019
02.18.2019
02.19.2019
02.20.2019
02.21.2019
02.22.2019
02.23.2019
02.24.2019
02.25.2019
02.26.2019
02.27.2019
02.28.2019
03.01.2019
03.02.2019
03.03.2019
03.04.2019
03.05.2019
03.06.2019
03.07.2019
03.08.2019
03.09.2019
03.10.2019
03.11.2019
03.12.2019
03.13.2019
03.14.2019
03.15.2019
03.16.2019
03.17.2019
03.18.2019
03.19.2019
03.20.2019
03.21.2019
03.22.2019
03.23.2019
03.24.2019
03.25.2019
03.26.2019
03.27.2019
03.28.2019
03.29.2019
03.30.2019
03.31.2019
04.01.2

In [ ]:
#list of gensets that can be run

#300's (except bus 313 and ZMC gens)
#['301_CT_1','301_CT_2','301_CT_3','301_CT_4','302_CT_1','302_CT_2','302_CT_3','302_CT_4','307_CT_1','307_CT_2'
#'315_STEAM_1','315_STEAM_2','315_STEAM_3','315_STEAM_4','315_STEAM_5',
#'315_CT_6','315_CT_7','315_CT_8','316_STEAM_1','318_CC_1'
#'321_CC_1','322_CT_5','322_CT_6','323_CC_1','323_CC_2']

#200's
#['201_CT_1','201_CT_2','201_STEAM_3','202_CT_1','202_CT_2','202_STEAM_3','202_STEAM_4','207_CT_1','207_CT_2','213_CC_3'
#'213_CT_1','213_CT_2','215_CT_4','215_CT_5','216_STEAM_1','218_CC_1','221_CC_1','223_STEAM_1','223_STEAM_2','223_STEAM_3'
#'223_CT_4','223_CT_5','223_CT_6']


In [818]:
gt = ['CT','STEAM','CC','NUCLEAR','HYDRO','SYNC_COND','RTPV','WIND','PV','CSP']
case = CreateRTSCase(gt,gen_data,storage_data,hydro_data,csp_data,rtpv_data,pv_data,wind_data,
                         bus_data,branch_data,reg_up_data,reg_down_data,flex_up_data,flex_down_data,
                         f,'1.3.2019.3.4hour',60,hours=4)
case.create_index()
case.pjm_generators('PJM_generators')
case.pjm_generators_full('PJM_generators_full')
case.storage('storage_resources')
case.init_gens('initialize_generators')
case.scheduled_gens('PJM_generators_scheduled_outage')
case.timepoints('timepoints_index')
case.zones('zones')
case.zonal_loads('timepoints_zonal')
case.tx_lines('transmission_lines')
case.tx_lines_hourly('transmission_lines_hourly',flow_multiplier=1)#.75
case.gs()
case.gs_seg() 

generator_segment  segment_length  marginal_cost  prev_offer  \
time Gen_Index                                                                  
1    101_CT_1                    0             0.4      97.863926   97.863926   
     101_CT_1                    1             0.2      97.863926   97.863926   
     101_CT_1                    2             0.2      98.070914   98.070914   
     101_CT_1                    3             0.2     107.136989  107.136989   
     101_CT_2                    0             0.4      97.863926   97.863926   
...                            ...             ...            ...         ...   
4    303_WIND_1                  3             0.0       0.000000    0.000000   
     122_WIND_1                  0             1.0       0.000000    0.000000   
     122_WIND_1                  1             0.0       0.000000    0.000000   
     122_WIND_1                  2             0.0       0.000000    0.000000   
     122_WIND_1                  3             0.0       0.000000    0.000000   

                 marginal_CO2  CO2damage  
time Gen_Index                            
1    101_CT_1        0.686267          0  
     101_CT_1        0.686267          0  
     101_CT_1        0.687718          0  
     101_CT_1        0.751294          0  
     101_CT_2        0.686267          0  
...                       ...        ...  
4    303_WIND_1      0.000000          0  
     122_WIND_1      0.000000          0  
     122_WIND_1      0.000000          0  
     122_WIND_1      0.000000          0  
     122_WIND_1      0.000000          0  

[2512 rows x 6 columns]

In [816]:
#s = time.time()
#this takes a bitCO2price=20
#v = case.gs_seg()
#print(round(time.time()-s,2))
#print(v)
print(v.abs())

                 generator_segment  segment_length  marginal_cost  prev_offer  \
time Gen_Index                                                                  
1    101_CT_1                  0.0             0.4      97.863926   97.863926   
     101_CT_1                  1.0             0.2      97.863926   97.863926   
     101_CT_1                  2.0             0.2      98.070914   98.070914   
     101_CT_1                  3.0             0.2     107.136989  107.136989   
     101_CT_2                  0.0             0.4      97.863926   97.863926   
...                            ...             ...            ...         ...   
24   303_WIND_1                3.0             0.0       0.000000    0.000000   
     122_WIND_1                0.0             1.0       0.000000    0.000000   
     122_WIND_1                1.0             0.0       0.000000    0.000000   
     122_WIND_1                2.0             0.0       0.000000    0.000000   
     122_WIND_1             

# create files

will go 1-by-1

In [701]:

#1: case index
case_index_df = pd.DataFrame({'genco':[1],
                            '':['index']})
case_index_df.set_index([''], inplace=True)
case_index_df.to_csv(os.path.join(f.RESULTS_INPUTS_DIRECTORY,'case_index.csv'))


In [702]:
#sets which generators to include
gentypes = ['CT','STEAM','CC','NUCLEAR','HYDRO','SYNC_COND','RTPV','WIND','PV','CSP']

In [703]:
#2: pjm_generators
pounds_to_metric_tonnes = 0.000453592
pjm_generators_dict = {}
index_list = ['Gen_Index','Capacity',"Fuel_Cost",'Pmin','start_cost','Can_Spin','Can_NonSpin',
                              'Min_Up','Min_Down','No_Load_Cost','Ramp_Rate',
              'tonneCO2perMWh','CO2price','CO2dollarsperMWh','ZoneLabel','GencoIndex']

pjm_generators_dict[index_list[0]] = gen_data[gen_data['Unit Type'].isin(gentypes)]['GEN UID'].values
pjm_generators_dict[index_list[1]] = gen_data[gen_data['Unit Type'].isin(gentypes)]['PMax MW'].values
gen_data['$/MWH'] = gen_data['Fuel Price $/MMBTU']*gen_data['HR_incr_3']/1000
pjm_generators_dict[index_list[2]] = gen_data[gen_data['Unit Type'].isin(gentypes)]['$/MWH'].values
#defaults for now
pjm_generators_dict[index_list[3]] = [0]*len(pjm_generators_dict[index_list[0]])
pjm_generators_dict[index_list[4]] = [0]*len(pjm_generators_dict[index_list[0]])
pjm_generators_dict[index_list[5]] = [1]*len(pjm_generators_dict[index_list[0]])
pjm_generators_dict[index_list[6]] = [1]*len(pjm_generators_dict[index_list[0]])
pjm_generators_dict[index_list[7]] = [1]*len(pjm_generators_dict[index_list[0]])
pjm_generators_dict[index_list[8]] = [1]*len(pjm_generators_dict[index_list[0]])
pjm_generators_dict[index_list[9]] = [0]*len(pjm_generators_dict[index_list[0]])

#ramp rates
pjm_generators_dict[index_list[10]] = 60*gen_data[gen_data['Unit Type'].isin(gentypes)]['Ramp Rate MW/Min'].values

gen_data['CO2/MWH'] = gen_data['Emissions CO2 Lbs/MMBTU']*pounds_to_metric_tonnes*gen_data['HR_incr_3']/1000
pjm_generators_dict[index_list[11]] = gen_data[gen_data['Unit Type'].isin(gentypes)]['CO2/MWH'].values
pjm_generators_dict[index_list[12]] = [0]*len(pjm_generators_dict[index_list[0]])
pjm_generators_dict[index_list[13]] = [a*b for a,b in zip(pjm_generators_dict[index_list[11]],pjm_generators_dict[index_list[12]])]
pjm_generators_dict[index_list[14]] = gen_data[gen_data['Unit Type'].isin(gentypes)]['Bus ID'].values
pjm_generators_dict[index_list[15]] = [2]*len(pjm_generators_dict[index_list[0]])

pjm_generators_df = pd.DataFrame.from_dict(pjm_generators_dict)
pjm_generators_df.set_index(['Gen_Index'],inplace=True)
pjm_generators_df.to_csv(os.path.join(f.RESULTS_INPUTS_DIRECTORY,'PJM_generators.csv'))

In [704]:
#2a pjm generators labeling
pjm_generators_full_dict = {}
index_list = ['Gen_Index','Name','Zone','Category','Capacity',
              'Fuel_Cost','Can_Spin','UTILUNIT']

pjm_generators_full_dict[index_list[0]] = pjm_generators_dict['Gen_Index']
pjm_generators_full_dict[index_list[1]] = pjm_generators_dict['Gen_Index']
pjm_generators_full_dict[index_list[2]] = pjm_generators_dict['ZoneLabel']
pjm_generators_full_dict[index_list[3]] = gen_data[gen_data['Unit Type'].isin(gentypes)]['Category'].values
pjm_generators_full_dict[index_list[4]] = pjm_generators_dict['Capacity']
pjm_generators_full_dict[index_list[5]] = pjm_generators_dict['Fuel_Cost']
pjm_generators_full_dict[index_list[6]] = pjm_generators_dict['Can_Spin']
pjm_generators_full_dict[index_list[7]] = ['NA']*len(pjm_generators_full_dict[index_list[0]])


pjm_generators_full_df = pd.DataFrame.from_dict(pjm_generators_full_dict)
#print(pjm_generators_full_df['Category'].values)

pjm_generators_full_df.set_index(['Gen_Index'],inplace=True)
pjm_generators_full_df.to_csv(os.path.join(f.RESULTS_INPUTS_DIRECTORY,'PJM_generators_full.csv'))

In [705]:
#3 storage resources

storage_dict = {}
index_list = ['Storage_Index','Discharge','Charge','SOCMax','DischargeEff','ChargeEff','StorageZoneLabel','StorageIndex']

storage_dict[index_list[0]] = gen_data[gen_data['Unit Type']=='STORAGE'].reset_index().at[0,'GEN UID']
storage_dict[index_list[1]] = gen_data[gen_data['Unit Type']=='STORAGE'].reset_index().at[0,'PMax MW']
storage_dict[index_list[2]] = gen_data[gen_data['Unit Type']=='STORAGE'].reset_index().at[0,'PMax MW']
storage_dict[index_list[3]] = storage_data.at[1,'Max Volume GWh']*1000
storage_dict[index_list[4]] = 1
storage_dict[index_list[5]] = 1
storage_dict[index_list[6]] = gen_data[gen_data['Unit Type']=='STORAGE'].reset_index().at[0,'Bus ID']
storage_dict[index_list[7]] = 2

#print(storage_dict)
storage_df = pd.DataFrame(storage_dict,index=[0])
storage_df.set_index(['Storage_Index'],inplace=True)
storage_df.to_csv(os.path.join(f.RESULTS_INPUTS_DIRECTORY,'storage_resources.csv'))

In [706]:
#4 init generators
init_dict = {}
index_list = ['Gen_Index','commit_init','time_up_init','time_down_init']

init_dict[index_list[0]] = pjm_generators_dict['Gen_Index']
init_dict[index_list[1]] = [1]*len(init_dict[index_list[0]])
init_dict[index_list[2]] = [200]*len(init_dict[index_list[0]])
init_dict[index_list[3]] = [0]*len(init_dict[index_list[0]])

init_df = pd.DataFrame.from_dict(init_dict)
init_df.set_index(['Gen_Index'],inplace=True)
init_df.to_csv(os.path.join(f.RESULTS_INPUTS_DIRECTORY,'initialize_generators.csv'))

In [707]:
#set # of hours to use
#hour_begin = 4928
hour_begin = 4920
#hour_begin = 0
hours = 24

In [708]:
#5 scheduled generation availability
#this **INCLUDES** capacity factor modification for availability of wind/solar/hydro
scheduled_dict = {}
index_list = ['timepoint','Gen_Index','available','Capacity',"Fuel_Cost"]

scheduled_dict[index_list[0]] = [e for e in list(range(1,hours+1)) for i in range(len(pjm_generators_dict['Gen_Index']))] 
scheduled_dict[index_list[1]] = list(pjm_generators_dict['Gen_Index'])*hours
scheduled_dict[index_list[2]] = [1]*len(scheduled_dict[index_list[1]])

gen_cap_list = []
#subset on first XXX hours for now but should eventually pull any subset
for h in range(hour_begin,hour_begin+hours):
    for gen,capacity in zip(pjm_generators_dict['Gen_Index'],pjm_generators_dict['Capacity']):
        if gen in hydro_data.columns:
            gen_cap_list.append(hydro_data.at[h,gen])
        elif gen in pv_data.columns:
            gen_cap_list.append(pv_data.at[h,gen])
        elif gen in rtpv_data.columns:
            gen_cap_list.append(rtpv_data.at[h,gen])
        elif gen in csp_data.columns:
            gen_cap_list.append(csp_data.at[h,gen])
        elif gen in wind_data.columns:
            gen_cap_list.append(wind_data.at[h,gen])
        else:
            gen_cap_list.append(capacity)
scheduled_dict[index_list[3]] = gen_cap_list#list(pjm_generators_dict['Capacity'])*hours

scheduled_dict[index_list[4]] = list(pjm_generators_dict['Fuel_Cost'])*hours

scheduled_df = pd.DataFrame.from_dict(scheduled_dict)
scheduled_df.set_index(['timepoint'],inplace=True)
scheduled_df.to_csv(os.path.join(f.RESULTS_INPUTS_DIRECTORY,'PJM_generators_scheduled_outage.csv'))

scheduled_df.tail()

,Gen_Index,available,Capacity,Fuel_Cost
timepoint,,,,
24,213_RTPV_1,1,0.0,0.0
24,309_WIND_1,1,8.9,0.0
24,317_WIND_1,1,62.9,0.0
24,303_WIND_1,1,77.3,0.0
24,122_WIND_1,1,2.3,0.0


In [709]:
#6 timepoints
timepoint_dict = {}
index_list = ['timepoint','reference_bus','reg_up_mw','reg_down_mw',
             'flex_up_mw','flex_down_mw']

timepoint_dict[index_list[0]] = list(range(1,hours+1))
timepoint_dict[index_list[1]] = [bus_data.at[12,'Bus ID']]*len(timepoint_dict[index_list[0]])

#reformatting for timepoints
period_list = []
for h in range(hour_begin,hour_begin+hours):
    if h==hour_begin:
        month = hydro_data.at[h,'Month']
        day = hydro_data.at[h,'Day']
    period_list.append(str(hydro_data.at[h,'Period']))
timepoint_dict[index_list[2]] = list(reg_up_data[(reg_up_data['Month']==month) & (reg_up_data['Day']==day)][period_list].values.ravel())
timepoint_dict[index_list[3]] = list(reg_down_data[(reg_down_data['Month']==month) & (reg_down_data['Day']==day)][period_list].values.ravel())
timepoint_dict[index_list[4]] = list(flex_up_data[(flex_up_data['Month']==month) & (flex_up_data['Day']==day)][period_list].values.ravel())
timepoint_dict[index_list[5]] = list(flex_down_data[(flex_down_data['Month']==month) & (flex_down_data['Day']==day)][period_list].values.ravel())


timepoint_df = pd.DataFrame.from_dict(timepoint_dict)
timepoint_df.set_index(['timepoint'],inplace=True)
timepoint_df.to_csv(os.path.join(f.RESULTS_INPUTS_DIRECTORY,'timepoints_index.csv'))

In [710]:
#load_data[(load_data['Month']==7) & (load_data['Day']==24)]

In [711]:
#7 zones
import math
zone_dict = {}
index_list = ['zone','wind_cap','solar_cap','voltage_angle_max','voltage_angle_min']

zone_dict[index_list[0]] = bus_data.loc[:,'Bus ID'].values
zone_dict[index_list[1]] = [0]*len(zone_dict[index_list[0]]) #eventually replace this
zone_dict[index_list[2]] = [0]*len(zone_dict[index_list[0]]) #eventually replace this
zone_dict[index_list[3]] = [180]*len(zone_dict[index_list[0]]) #math.pi/3
zone_dict[index_list[4]] = [-180]*len(zone_dict[index_list[0]]) #math.pi/3

zone_df = pd.DataFrame.from_dict(zone_dict)
zone_df.set_index(['zone'],inplace=True)
zone_df.to_csv(os.path.join(f.RESULTS_INPUTS_DIRECTORY,'zones.csv'))

In [712]:
#create hourly nodal loads
#load_data.loc[:23]
bus_df = bus_data[['Bus ID','MW Load','Area']]
bus_df_load = pd.merge(bus_df,bus_df.groupby('Area').sum()[['MW Load']].reset_index(),how='left',left_on='Area',right_on='Area')
bus_df_load['Frac Load'] = bus_df_load['MW Load_x']/bus_df_load['MW Load_y']
hourly_df = pd.concat([bus_df_load]*hours, ignore_index=True)
hourly_df['timepoint'] = [e for e in timepoint_dict['timepoint'] for i in zone_dict['zone']]
#print(load_data)
load_short = load_data.loc[hour_begin:hour_begin+hours-1].set_index(['Month','Day','Period']).reset_index() #sets unique index
hourly_df['zonal_load'] = [load_short.at[t-1,str(a)] for t,a in zip(hourly_df['timepoint'].values,hourly_df['Area'].values)]
hourly_df['bus load'] = hourly_df['zonal_load']*hourly_df['Frac Load']
hourly_df.head()

,Bus ID,MW Load_x,Area,MW Load_y,Frac Load,timepoint,zonal_load,bus load
0,101,108.0,1,2850.0,0.037895,1,1603.036863,60.746660
1,102,97.0,1,2850.0,0.034035,1,1603.036863,54.559500
2,103,180.0,1,2850.0,0.063158,1,1603.036863,101.244433
3,104,74.0,1,2850.0,0.025965,1,1603.036863,41.622712
4,105,71.0,1,2850.0,0.024912,1,1603.036863,39.935304


In [713]:
#8 timepoints zonal
time_zone_dict = {}
index_list = ['timepoint','zone','gross_load','wind_cf','solar_cf']

time_zone_dict[index_list[0]] = [e for e in timepoint_dict['timepoint'] for i in zone_dict['zone']] 
time_zone_dict[index_list[1]] = [i for e in timepoint_dict['timepoint'] for i in zone_dict['zone']]
time_zone_dict[index_list[2]] = hourly_df['bus load'].values#hourly_df['MW Load_x'].values
time_zone_dict[index_list[3]] = [0]*len(time_zone_dict[index_list[0]])
time_zone_dict[index_list[4]] = [0]*len(time_zone_dict[index_list[0]])

time_zone_df = pd.DataFrame.from_dict(time_zone_dict)
time_zone_df.set_index(['timepoint'],inplace=True)
time_zone_df.to_csv(os.path.join(f.RESULTS_INPUTS_DIRECTORY,'timepoints_zonal.csv'))

In [714]:
#update branch data
baseMVA = 100
km_per_mile = 1.60934
branch_data_update = pd.merge(branch_data,bus_data[['Bus ID','BaseKV']],how='left',left_on='From Bus',right_on='Bus ID')
branch_data_update['puconversion'] = branch_data_update.BaseKV**2/baseMVA #V^2/P=R, kv^2 and MV will cancel units
branch_data_update['Lengthkm'] =branch_data_update['Length']*km_per_mile 
for i in branch_data_update.index:
    if branch_data_update.at[i,'B']==0:
        branch_data_update.at[i,'puconversion'] = branch_data_update.at[i,'Tr Ratio']
        branch_data_update.at[i,'Lengthkm'] = 1
    else:
        branch_data_update.at[i,'puconversion'] = 1
branch_data_update['x_final']=branch_data_update.X*branch_data_update.puconversion#*branch_data_update.Lengthkm
branch_data_update['r_final']=branch_data_update.R*branch_data_update.puconversion#*branch_data_update.Lengthkm
branch_data_final = branch_data_update#[branch_data_update['Lengthkm']!=0].copy()
branch_data_update.loc[:7,:]

,UID,From Bus,To Bus,R,X,B,Cont Rating,LTE Rating,STE Rating,Perm OutRate,Duration,Tr Ratio,Tran OutRate,Length,Bus ID,BaseKV,puconversion,Lengthkm,x_final,r_final
0,A1,101,102,0.003,0.014,0.461,175,193,200,0.24,16,0.000,0.0,3.0,101,138.0,1.000,4.82802,0.01400,0.00300
1,A2,101,103,0.055,0.211,0.057,175,208,220,0.51,10,0.000,2.9,55.0,101,138.0,1.000,88.51370,0.21100,0.05500
2,A3,101,105,0.022,0.085,0.023,175,208,220,0.33,10,0.000,1.2,22.0,101,138.0,1.000,35.40548,0.08500,0.02200
3,A4,102,104,0.033,0.127,0.034,175,208,220,0.39,10,0.000,1.7,33.0,102,138.0,1.000,53.10822,0.12700,0.03300
4,A5,102,106,0.050,0.192,0.052,175,208,220,0.48,10,0.000,2.6,50.0,102,138.0,1.000,80.46700,0.19200,0.05000
5,A6,103,109,0.031,0.119,0.032,175,208,220,0.38,10,0.000,1.6,31.0,103,138.0,1.000,49.88954,0.11900,0.03100
6,A7,103,124,0.002,0.084,0.000,400,510,600,0.02,768,1.015,0.0,0.0,103,138.0,1.015,1.00000,0.08526,0.00203
7,A8,104,109,0.027,0.104,0.028,175,208,220,0.36,10,0.000,1.4,27.0,104,138.0,1.000,43.45218,0.10400,0.02700


In [715]:
#create susceptance value for DCOPF
#this will have to be checked more at some point
#add factor of 1000000 b/c want MB to get MW, not B to get W
#branch_data_final['Susceptance'] = [1*(x)/((x)**2+(r)**2) for x,r,length in zip(branch_data_final['x_final'].values,branch_data_final['r_final'].values,branch_data_final['Lengthkm'].values)]
branch_data_final['Susceptance'] = [1/x*100*math.pi/180 for x in branch_data_final['x_final'].values]
#branch_data_final.to_csv('check.csv')
#

In [716]:
#9 transmission lines
tx_dict = {}
index_list = ['transmission_line','susceptance']

tx_dict[index_list[0]] = branch_data_final['UID'].values
tx_dict[index_list[1]] = branch_data_final['Susceptance'].values 

tx_df = pd.DataFrame.from_dict(tx_dict)
tx_df.set_index(['transmission_line'],inplace=True)
tx_df.to_csv(os.path.join(f.RESULTS_INPUTS_DIRECTORY,'transmission_lines.csv'))

In [717]:
#10 transmission lines hourly
tx_hourly_dict = {}
index_list = ['timepoint','transmission_line','transmission_from','transmission_to',
              'min_flow','max_flow','hurdle_rate']

tx_hourly_dict[index_list[0]] = [t for t in timepoint_dict['timepoint'] for line in tx_dict['transmission_line']]
tx_hourly_dict[index_list[1]] = [line for t in timepoint_dict['timepoint'] for line in tx_dict['transmission_line']]
tx_hourly_dict[index_list[2]] = [t_from for t in timepoint_dict['timepoint'] for t_from in branch_data_final['From Bus'].values]
tx_hourly_dict[index_list[3]] = [t_to for t in timepoint_dict['timepoint'] for t_to in branch_data_final['To Bus'].values]
tx_hourly_dict[index_list[4]] = [-flow for t in timepoint_dict['timepoint'] for flow in branch_data_final['Cont Rating'].values]
tx_hourly_dict[index_list[5]] = [flow for t in timepoint_dict['timepoint'] for flow in branch_data_final['Cont Rating'].values]
tx_hourly_dict[index_list[6]] = [0]*len(tx_hourly_dict[index_list[0]])

tx_hourly_df = pd.DataFrame.from_dict(tx_hourly_dict)
tx_hourly_df.set_index(['timepoint','transmission_line'],inplace=True)
tx_hourly_df.to_csv(os.path.join(f.RESULTS_INPUTS_DIRECTORY,'transmission_lines_hourly.csv'))

In [718]:
#11 gen segments
#this should become defunct
gs_list = [0,1,2,3]
gs_df = pd.DataFrame({'generator_segment':gs_list,
                            'length':[.4,.2,.2,.2]})
gs_df.set_index(['generator_segment'], inplace=True)
gs_df.to_csv(os.path.join(f.RESULTS_INPUTS_DIRECTORY,'generator_segments.csv'))
#create generator-wise segments


In [719]:
#12 gen segment mc
gs_seg_dict = {}
index_list = ['time','Gen_Index','generator_segment','segment_length',
              'marginal_cost','prev_offer','marginal_CO2']

gs_seg_dict[index_list[0]] = [t for t in timepoint_dict['timepoint'] for gen in pjm_generators_dict['Gen_Index'] for gs in gs_list]
gs_seg_dict[index_list[1]] = [gen for t in timepoint_dict['timepoint'] for gen in pjm_generators_dict['Gen_Index'] for gs in gs_list]
gs_seg_dict[index_list[2]] = [gs for t in timepoint_dict['timepoint'] for gen in pjm_generators_dict['Gen_Index'] for gs in gs_list]
# = [ for gen,gs in zip(gs_seg_dict[index_list[1]],gs_seg_dict[index_list[2]])]

mcos_list = []
emissions_list = []
seg_length_list = []
for gen,gs in zip(gs_seg_dict[index_list[1]],gs_seg_dict[index_list[2]]):
    EmissionsRate = gen_data.set_index('GEN UID').at[gen,'Emissions CO2 Lbs/MMBTU']*pounds_to_metric_tonnes
    if gs==0:
        seg_length_list.append(gen_data.set_index('GEN UID').at[gen,'Output_pct_'+str(gs)])
        if gen_data.set_index('GEN UID').at[gen,'HR_incr_'+str(1)]/1000!=0:
            HeatRate = gen_data.set_index('GEN UID').at[gen,'HR_incr_'+str(1)]/1000 #convexify offer curve
        else:
            HeatRate = gen_data.set_index('GEN UID').at[gen,'HR_avg_'+str(gs)]/1000
        mcos_list.append(HeatRate*gen_data.set_index('GEN UID').at[gen,'Fuel Price $/MMBTU'])
        emissions_list.append(HeatRate*EmissionsRate)
    else:
        seg_length_list.append(max(0,gen_data.set_index('GEN UID').at[gen,'Output_pct_'+str(gs)]-gen_data.set_index('GEN UID').at[gen,'Output_pct_'+str(gs-1)]))
        if gen_data.set_index('GEN UID').at[gen,'HR_incr_'+str(gs)]/1000==0 and gen_data.set_index('GEN UID').at[gen,'HR_avg_'+str(0)]/1000 !=0:
            HeatRate = gen_data.set_index('GEN UID').at[gen,'HR_avg_'+str(0)]/1000
        else:
            HeatRate = gen_data.set_index('GEN UID').at[gen,'HR_incr_'+str(gs)]/1000
        mcos_list.append(HeatRate*gen_data.set_index('GEN UID').at[gen,'Fuel Price $/MMBTU'])
        emissions_list.append(HeatRate*EmissionsRate)

gs_seg_dict[index_list[3]] = seg_length_list
gs_seg_dict[index_list[4]] = mcos_list
gs_seg_dict[index_list[5]] = gs_seg_dict[index_list[4]]
#Emissions CO2 Lbs/MMBTU
#gen_data['CO2/MWH']
gs_seg_dict[index_list[6]] = emissions_list
#gs_seg_dict[index_list[4]] = [mc for t in timepoint_dict['timepoint'] for mc in pjm_generators_dict['Fuel_Cost']]
#gs_seg_dict[index_list[5]] = gs_seg_dict[index_list[4]] #sets offer to mc  
#gs_seg_dict[index_list[6]]

gs_seg_df = pd.DataFrame.from_dict(gs_seg_dict)
gs_seg_df.set_index(['time','Gen_Index'],inplace=True)
#overwrite 0's on segment 1 for renewable generators
gs_seg_df.loc[(gs_seg_df.generator_segment == 0) & (gs_seg_df.segment_length == 0), 'segment_length'] = 1
gs_seg_df.to_csv(os.path.join(f.RESULTS_INPUTS_DIRECTORY,'generator_segment_marginalcost.csv'))